In [1]:
import numpy as np
import random
#np.random.seed(123)

In [2]:
def areBinaryWeights(weights):
    flag = True
    th = 1e-12
    for i in range(np.shape(weights),-1,-1):
        if(weights[i] > th and (1-weights[i])>th):
            flag = false
            break
            
    return flag

In [3]:
import uuid
import datetime

def empty_problem():
    problem = {}
    problem['type'] = 'None'
    problem['N'] = 0
    problem['outliers'] = []
    problem['priors'] = []
    problem['dof'] = 0
    #problem['time'] = now
    problem['time'] = datetime.datetime.now()
    # Unique identifier
    problem['uuid'] = uuid.uuid4().hex
    return problem
    

In [4]:
def isProblem(problem):
    mandatory_fields = {'type', 'N', 'outliers', 'priors', 'dof'}
    #flag = all(isfield(problem, mandatory_fields))
    flag = all(field in problem for field in mandatory_fields)
    #flag = all(field in problem for field in mandatory_fields)
    return flag

In [5]:
def rand_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random. rand(n,m)
    
    return r

In [6]:
def randn_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random.randn(n,m)
    return r

In [7]:
from numpy.linalg import matrix_rank
import random

def random_matrix(n, m, desired_rank):
    max_rank = min(n, m)
    if desired_rank==None:
        desired_rank = max_rank
    assert desired_rank<=max_rank, 'Cannot generate a matrix a random matrix with the desired rank'
    M = 3*np.random.rand(n, m)
    while matrix_rank(M) < max_rank and matrix_rank(M) > 0:
        M = rand_in(n, m, [-1, 1])
    rank_M = matrix_rank(M)
    
    if rank_M != desired_rank:
        col_perm = random.sample(range(m), rank_M - desired_rank + 1)
        for i in range(rank_M-desired_rank+1):
                M[:, col_perm[0]] = M[:, col_perm[i]]
    return M

In [8]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    th1 = (mu+1)/mu*barc2
    th2 = (mu)/(mu+1)*barc2
    #print('mu is: ', mu)
    #print('th1 is: ', th1)
    #print('th2 is: ', th2)
    
    for k in range(np.shape(residuals)):
        if residuals[k] - th1 >=0:
            weights[k] = 0
        elif residuals[k] - th2 <=0:
            weights[k] = 1
        else:
            weights[k] = np.sqrt(barc2*mu*(mu+1)/residuals[k]) - mu
            #assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!', weights(k)
            assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!'
  
    return weights, th1, th2     
    

In [9]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
import numpy as np
import numbers
import random
from scipy.stats import chi2

In [10]:
def linearRegressionProblem(N, outliers_percentage, **kwargs):
    
    StateDimension = kwargs.get('StateDimension', 3)
    #print(StateDimension, type(StateDimension))
    #print(isinstance(StateDimension, numbers.Number))
    #print(np.size(StateDimension), np.size(StateDimension)==1) 
    #print(StateDimension >=1) 
    
    assert isinstance(StateDimension, numbers.Number) and np.size(StateDimension)==1 and StateDimension >=1, 'StateDimension must be numeric, scalar, and >=1'
    
    MeasurementDimension = kwargs.get('MeasurementDimension', 3)
    assert isinstance(MeasurementDimension, numbers.Number) and np.size(MeasurementDimension)==1 and np.array(MeasurementDimension)>=1, 'MeasurementDimension must be numeric, scalar, and >=1'
    
    StateMeanandStd = kwargs.get('StateMeanAndStd', [1,1])
    assert all(isinstance(elem, numbers.Number) for elem in StateMeanandStd) and np.size(StateMeanandStd)==2 and all(np.array(StateMeanandStd) >=0), 'StateMeanandStd should be numeric, vector of length 2, and >=0'
    
    MeasurementNoiseStd = kwargs.get('MeasurementNoiseStd', 0.1)
    assert isinstance(MeasurementNoiseStd, numbers.Number) and np.size(MeasurementNoiseStd)==1 and MeasurementNoiseStd>0
    
    OutlierSpace = kwargs.get('OutlierSpace', [-100, 100])
    assert all(isinstance(elem, numbers.Number) for elem in OutlierSpace) and np.size(OutlierSpace)==2 and np.array(OutlierSpace[0]) <= np.array(OutlierSpace[1]), 'OutlierSpace must be numeric, vector of length 2, and OutlierSpace[0] < OutlierSpace[1]'
    
    if kwargs==None:
        outliers_percentage = 0
        
    problem = empty_problem()
    problem['N'] = N
    problem['StateDimension'] = StateDimension
    problem['MeasurementDimension'] = MeasurementDimension
    problem['MeasurementNoiseStd'] = MeasurementNoiseStd
    problem['dof'] = MeasurementDimension
    problem['type'] = 'linear-estimation'
    num_outliers = np.floor(outliers_percentage*N)
    problem['x_gt'] = np.random.normal(StateMeanandStd[0], StateMeanandStd[1], (StateDimension,1))
    
    problem['A'] = []
    for i in range(N):
        problem['A'].append(random_matrix(MeasurementDimension, StateDimension, min(MeasurementDimension, StateDimension)))
    #print('MeasurementDimension: ', MeasurementDimension)
    problem['y'] = []
    for i in range(N):
        problem['y'].append(problem['A'][i] .dot( problem['x_gt']) +  np.random.normal(0, int(MeasurementNoiseStd), size=(int(MeasurementDimension), 1)))
    
    convi = 1/MeasurementNoiseStd*np.eye(MeasurementDimension)
    problem['outliers'] = sorted(random.sample(range(problem['N']), int(np.array(num_outliers))))
    
    for i in problem['outliers']:
        y_gt = problem['y'][i]
        d = 0
        #print(OutlierSpace, np.shape(OutlierSpace))
        #print(d, chi2.ppf(0.999, int(MeasurementDimension)), d < chi2.ppf(0.999, int(MeasurementDimension)))
        while (d < chi2.ppf(0.999, int(MeasurementDimension))):
            #y_meas = rand_in(MeasurementDimension, 1, range(OutlierSpace[0], OutlierSpace[1]+1))
            
            y_meas = rand_in(MeasurementDimension, 1, (OutlierSpace))

            d = (y_gt - y_meas).T @ convi @ (y_gt - y_meas)
            #print(d < chi2.ppf(0.999, int(MeasurementDimension)))
        
        problem['y'][i] = y_meas
        
    problem['priors'] = []
    
    return problem
        
    
    

In [11]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);


In [12]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    #print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    #print('Weights: ', Weights)
    #print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    #print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    #print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [13]:
problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

In [14]:
def printProblemSummary(problem):
    num_outliers = np.size(problem['outliers']);
    print('Problem:\n');
    print('Type:', problem['type']);
    print('Measurements (N):', problem['N']);
    print('Num. outliers (k): ', num_outliers);
    print('Num. priors:   ', len(problem['priors']));
    print('Num. possible outliers (|V|): ', problem['N'] - np.size(problem['priors']));
    print('   k/N: %.03f\n', num_outliers/problem['N']);
    print('   k/|V|: %.03f\n', num_outliers/(problem['N'] - len(problem['priors'])));
    

In [15]:
printProblemSummary(problem)

Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6


In [16]:
print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

  - Least-Square



In [17]:
print(f_val_ls)

640441.7201253088


In [18]:
import numpy as np
from scipy.stats import chi2

cfg                 = {}
cfg['epsilon']      = chi2.ppf(0.99, problem['dof']) * np.array(problem['MeasurementNoiseStd'])**2
cfg['epsilon_mts']  = lambda n: chi2.ppf(0.99, n*problem['dof'])
cfg['epsilon_dmts'] = lambda n1, n2: UDGinv(0.99, n1*problem['dof']/2, n2*problem['dof']/2, 2*problem['MeasurementNoiseStd']**2, 2*problem['MeasurementNoiseStd']**2)
results             = {}
print('Solving with:')

Solving with:


In [19]:
print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime


Solving with:



In [20]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    #print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    #print('Weights: ', Weights)
    #print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    #print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    """"
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    """
    
    if 'Estimate' in kwargs and kwargs['Estimate'] is not None:
        x = kwargs['Estimate']
    else:
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    #print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [21]:
print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

  - Ground-Truth Least-Square...



In [22]:
start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

In [23]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))

[ 4  7  9 13 14 18 24 27 28 31 32 33 34 42 45 48 50 60 61 64 65 66 67 68
 69 73 74 75 77 78 83 84 85 86 87 91 92 93 94 97]
(array([ 4,  7,  9, 13, 14, 18, 24, 27, 28, 31, 32, 33, 34, 42, 45, 48, 50,
       60, 61, 64, 65, 66, 67, 68, 69, 73, 74, 75, 77, 78, 83, 84, 85, 86,
       87, 91, 92, 93, 94, 97], dtype=int64),)


In [24]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))
print(info_gt['residuals']<8)

[ 4  7  9 13 14 18 24 27 28 31 32 33 34 42 45 48 50 60 61 64 65 66 67 68
 69 73 74 75 77 78 83 84 85 86 87 91 92 93 94 97]
(array([ 4,  7,  9, 13, 14, 18, 24, 27, 28, 31, 32, 33, 34, 42, 45, 48, 50,
       60, 61, 64, 65, 66, 67, 68, 69, 73, 74, 75, 77, 78, 83, 84, 85, 86,
       87, 91, 92, 93, 94, 97], dtype=int64),)
[False False False False  True False False  True False  True False False
 False  True  True False False False  True False False False False False
  True False False  True  True False False  True  True  True  True False
 False False False False False False  True False False  True False False
  True False  True False False False False False False False False False
  True  True False False  True  True  True  True  True  True False False
 False  True  True  True False  True  True False False False False  True
  True  True  True  True False False False  True  True  True  True False
 False  True False False]


In [25]:
print(f_val_ls)
print(f_val_gt)

640441.7201253088
1.0944674687963784e-27


In [26]:
def detectionStats(problem, inliers):
    num_outliers = len(problem['outliers'])
    if num_outliers == 0:
        true_positive_rate = 1
        false_positive_rate = 0
        precision = 1
        recall = 1
    else:
        outliers = np.setdiff1d(np.arange(1, problem['N']+1), inliers)
        true_inliers = np.setdiff1d(np.arange(1, problem['N']+1), np.union1d(problem['priors'], problem['outliers']))
        if len(true_inliers) == 0:
            true_positive_rate = 0
            false_positive_rate = 0
            precision = 0
            recall = 0
        else:
            true_positive = np.intersect1d(problem['outliers'], outliers)
            false_positive = np.intersect1d(true_inliers, outliers)
            true_positive_rate = len(true_positive) / num_outliers
            false_positive_rate = len(false_positive) / len(true_inliers)
            precision = len(true_positive) / (len(true_positive) + len(false_positive))
            recall = len(true_positive) / num_outliers
    return true_positive_rate, false_positive_rate, precision, recall


In [27]:
#tp, fp = detectionStats(problem, range(problem['N']))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}

In [28]:
detectionStats(problem, range(problem['N']))

(0.0, 0.024390243902439025, 0.0, 0.0)

In [29]:
results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

In [30]:
print(results['gt']['f_val'])
print(results['ls']['f_val'])

1.0944674687963784e-27
640441.7201253088


In [31]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    # GNC - GNC weights update for the TLS cost function
    # Reference:
    #    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    #      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    #      https://arxiv.org/pdf/1909.08605.pdf
    #    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications"
    #      Antonante, Tzoumas, Yang, Carlone (2020).
    #      https://arxiv.org/pdf/2007.15109.pdf

    # Author: Pasquale Antonante
    # email: antonap@mit.edu
    # Date: 2021-01-06

    th1 = (mu+1)/mu * barc2
    th2 = (mu)/(mu+1) * barc2  # th1 > th2
    
    #print('th1 and th2 are: \n', th1, th2)
    
    #display(np.shape(np.array(residuals)))
    #display(np.size(np.array(residuals)))
    
    for k in range(np.size(residuals)):
    #for k in range(np.shape(residuals)[1]):
        #print('Residual[k]', residuals[k])
        #residuals_k = np.squeeze(residuals[k])
        #display(k, np.shape(residuals), np.shape(residuals[k]), np.shape(th1), np.shape(th2))
        #display(residuals[k])
        #display('Residuals[k] is: ', residuals[k])
        if (residuals[k] - th1) >= 0:
            weights[k] = 0
            #print('residuals(k) - th1 >= 0: \n', residuals[k], th1 )
        elif (residuals[k] - th2) <= 0:
            weights[k] = 1
            #print('residuals(k) - th2 <= 0: \n', residuals[k], th2 )
        else:
            weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu
            
            #print('weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu', residuals[k], weights[k])
            assert 0 <= weights[k] <= 1, f'weights {weights[k]} calculation wrong!'
    
    #display(type(weights))

    return weights, th1, th2


In [32]:
import math  
def gnc_vanilla(problem, f, **kwargs):
    continuation_factor = kwargs.get('ContinuationFactor', 1.4)
    initial_mu = kwargs.get('InitialMu', 'auto')
    noise_bound = kwargs.get('NoiseBound', 0)
    max_iterations = kwargs.get('MaxIterations', 1e3)
    fix_priors_weights = kwargs.get('FixPriorsWeights', True)
    cost_threshold = kwargs.get('CostThreshold', 0)
    debug = kwargs.get('Debug', False)
    init_ = kwargs.get('init_', 0)

    max_iterations = int(max_iterations)

    if debug:
        residuals_history = []
        weights_history = []

    if 'init_' not in kwargs:
        try:
            _, f_info = f(problem)
            #print('f(problem): ', f(problem))
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        assert 'residuals' in f_info, 'f should compute residuals'
    else:
        f_info = kwargs['init_']

    barc2 = noise_bound
    weights = np.ones(problem['N'])
    
    #print('barc2 is: {}', barc2)
    #print('Max of residuals: {}', np.max(f_info['residuals']))
    #print('Residuals are: {}', f_info['residuals'])
    #display(type(weights))
    
    
    if isinstance(initial_mu, str) and initial_mu.lower() == 'auto':
        mu = 1 / (2 * np.max(f_info['residuals']) / barc2 - 1 )
        #display('mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )')
    elif isinstance(initial_mu, (int, float)):
        mu = initial_mu
    else:
        raise ValueError('Invalid value for InitialMu')

    prev_f_cost = 0

    i = 1
    info = {}
    info['stopping'] = 'MaxIterations'  # Worst case stopping
    info['status'] = 1  # worst case status: failure

    while i < max_iterations:
        if debug:
            residuals_history.append(f_info['residuals'].tolist())
            weights_history.append(weights.tolist())
            
        #print('mu is: {}',mu)
        #print('f_info[residuals]')
        #print(f_info['residuals'])
        
        weights, th1out, th2out = gncWeightsUpdate(weights, mu, f_info['residuals'], barc2)
        #print('Weights after gnc algorithm')
        #print(weights)
        
        #print(type(weights))
        #print(type(list(weights)))
        
        if fix_priors_weights:
            weights[problem['priors']] = 1
        
        try:
            _, f_info = f(problem, Weights=weights)
            #print('_, f_info = f(problem, Weights=weights)')
            #print(weights)
            
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        
        f_cost = np.sum(f_info['residuals'] * weights)
        
        cost_diff = np.abs(f_cost - prev_f_cost)
        
        prev_f_cost = f_cost

        if (cost_diff < cost_threshold) or areBinaryWeights(weights):
            if debug:
                residuals_history.append(f_info['residuals'].tolist())
                weights_history.append(weights.tolist())
            info['residuals'] = f_info['residuals'].tolist()
            info['stopping'] = 'CostThreshold'
            info['status'] = 1
            break
        mu *= continuation_factor
        i += 1
    
    
    #print(np.where(weights > 1-2.2204e-16))
    
    inliers = np.where(weights > 1-2.2204e-16)[0].tolist()
    
    info['Iterations'] = i
    info['params'] = kwargs
    info['Algorithm'] = 'GNC'
    if debug:
        info['mu'] = mu
        info['barc2History'] = np.repeat(barc2, i+1)
        info['ResidualsHistory'] = residuals_history
        info['WeightsHistory'] = weights_history
        info['CostDiff'] = cost_diff
        info['AreBinaryWeights'] = areBinaryWeights(weights)

    return inliers, info


In [33]:
def gnc(problem, f, **kwargs):
    """
    %GNC - Graduated Non-Convexity
    % Implementation of: 
    %    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    %      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    %      https://arxiv.org/pdf/1909.08605.pdf
    %    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications" 
    %      Antonante, Tzoumas, Yang, Carlone (2020).
    %      https://arxiv.org/pdf/2007.15109.pdf
    %
    % Syntax:  [inliers, info] = gnc(problem, f, ...)
    %
    % Inputs:
    %    problem - the stracture representing a generic problem (see EmptyProblem)
    %    f - a function_handler of a non-minimal solver for problem
    %
    % Options:
    %    - ContinuationFactor: continuation factor for mu
    %    - InitialMu: initial mu (default: auto)
    %    - NoiseBound: inlier threshold
    %    - MaxIterations: maximum number of iterations
    %    - FixPriorsWeights: fix priors weights to 1 across all iterations
    %    - CostThreshold: if weighted sum of squared residuals is below this value, GNC stops
    %    - Debug: whether or not to enable the debug information
    %
    % Outputs:
    %    inliers - the indices of the inliers
    %    info - structure containing extended information about the xecution
    % 
    % Example:
    %   problem = linearRegressionProblem(100, 0.8);
    %   [inliers, info] = gnc(problem, @leastSquareNorm2, ...
    %       'NoiseBound', chi2inv(0.99, problem.dof)*problem.MeasurementNoiseStd^2);

    % Author: Pasquale Antonante
    % email: antonap@mit.edu
    % Date: 2021-01-06
    """
    
    #inliers = kwargs.get('Inliers', measurements)
    #assert all(isinstance(elem, numbers.Number) for elem in inliers) and all(np.floor(inliers)==inliers), "Inliers must be numeric and integers"
    #print(inliers)
    #weights = kwargs.get('Weights', default_weights)
    
    
    #params = inputParser;
    #params.KeepUnmatched = true;
    #params.addParameter('NoiseBound', 0, @(x) isscalar(x) && x>=0 && isfinite(x));
    #arams.parse(varargin{:});
    #start_t = now;
    #if params.Results.NoiseBound > 0
    #    [inliers, info] = gnc_vanilla(problem, f, varargin{:});
    #else
    #    error('You need to set the Noise Bound')
    #end
    #end_t = now;
    
    NoiseBound = kwargs.get('NoiseBound', 0)
    assert(np.size(NoiseBound)==1 and NoiseBound >= 0 and np.isfinite(NoiseBound))
    
    assert isProblem(problem), 'The problem doesn''t contain required fields.'

    start_t = datetime.datetime.now();
    
    if NoiseBound > 0:
        [inliers, info] = gnc_vanilla(problem, f, **kwargs);
    else:
        ValueError('You need to set the Noise Bound')
    
    #print('Output of gnc_vanilla')
    #print(inliers)
    end_t = datetime.datetime.now();
    
    info = {}
    info['time'] = (end_t - start_t) * 24 * 60 * 60; #% serial date number to sec
    
    return inliers, info
    


In [34]:
def areBinaryWeights(weights):
    # Check the input vector is (approximately) a binary vector
    flag = True
    th = 1e-12
    for i in range(len(weights)-1, -1, -1):
        if weights[i] > th and abs(1 - weights[i]) > th:
            flag = False
            break
    return flag

In [35]:
print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC



In [36]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

640441.7201253088
1.0944674687963784e-27
1.0944674687963784e-27


In [37]:
print(inliers_gnc)

[4, 7, 9, 13, 14, 18, 24, 27, 28, 31, 32, 33, 34, 42, 45, 48, 50, 60, 61, 64, 65, 66, 67, 68, 69, 73, 74, 75, 77, 78, 83, 84, 85, 86, 87, 91, 92, 93, 94, 97]


In [38]:
print(inliers_gt)

[ 4  7  9 13 14 18 24 27 28 31 32 33 34 42 45 48 50 60 61 64 65 66 67 68
 69 73 74 75 77 78 83 84 85 86 87 91 92 93 94 97]


In [39]:
print(inliers_gnc == inliers_gt)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [40]:
print(info_gt['residues'])

[[-6.91136557e+01  2.41710910e+01  9.91686209e+01 -2.57265892e+01
   2.38697950e-15 -7.90494145e+01 -7.71225627e+01 -6.66133815e-16
  -7.07236693e+01  0.00000000e+00  7.16817255e+01  4.74880580e+01
  -9.68023489e+01 -3.10862447e-15  4.44089210e-15  4.86503923e+01
  -7.98419054e+00 -9.11622665e+01  8.88178420e-16 -5.29135132e+01
   5.03982926e+01 -2.65710486e+01  6.21226260e+01  4.31349442e+01
   8.88178420e-16  7.96009726e+01  9.34501129e+01 -2.66453526e-15
   4.44089210e-16 -9.24542605e+01  3.91223533e-01 -8.88178420e-16
  -3.55271368e-15  6.10622664e-16 -6.21724894e-15 -4.26690657e+01
   9.90374716e+01 -4.53841740e+01  4.04505956e+00  3.72177183e+01
   5.16349270e+01 -6.95282691e+01  3.99680289e-15 -8.95923871e+01
   9.52357913e+01 -3.77475828e-15 -5.10048480e+01  1.33867255e+01
   6.66133815e-16  1.98392524e+00 -2.66453526e-15  2.24116904e+01
  -3.19481819e+01  6.41044588e+01  8.42593623e+01 -9.78656580e+01
   9.79256824e+01  5.46074761e+01 -8.40042410e+01 -4.67784577e+01
  -2.33146

In [41]:
print(info_gt['residuals'])

[1.36747582e+04 1.41125441e+04 1.61899310e+04 3.75070172e+03
 3.27161572e-29 1.45876906e+04 1.98560344e+04 2.21867130e-30
 7.22164052e+03 3.71750702e-29 9.43163751e+03 8.70799093e+03
 1.66923985e+04 3.80625387e-29 4.12179823e-29 3.11656307e+03
 5.38262914e+02 1.47396101e+04 7.06277029e-30 4.71736203e+03
 8.80327654e+03 1.02347985e+04 1.61790184e+04 2.47339704e+03
 3.31321580e-29 1.42524373e+04 1.28500554e+04 1.51855724e-29
 2.02145607e-30 1.24208899e+04 1.28438403e+04 4.89093761e-29
 2.37151310e-29 4.31716456e-30 6.67080503e-29 4.04861829e+03
 2.46605273e+04 1.01284708e+04 2.92006243e+03 9.51638650e+03
 8.26573508e+03 6.87124652e+03 2.56872832e-29 9.71505468e+03
 2.37915554e+04 4.65920972e-29 1.24730281e+04 2.27842191e+03
 9.03492256e-30 1.15798396e+04 2.87071414e-29 1.05009156e+04
 1.01185227e+04 2.25435491e+04 1.04169313e+04 1.77472768e+04
 1.74303619e+04 2.09527637e+04 1.30590790e+04 4.40481139e+03
 1.08098596e-29 3.48084874e-29 5.93397101e+03 1.61384438e+04
 2.01652569e-29 9.412096

In [42]:
print(info_gt['residuals'])

[1.36747582e+04 1.41125441e+04 1.61899310e+04 3.75070172e+03
 3.27161572e-29 1.45876906e+04 1.98560344e+04 2.21867130e-30
 7.22164052e+03 3.71750702e-29 9.43163751e+03 8.70799093e+03
 1.66923985e+04 3.80625387e-29 4.12179823e-29 3.11656307e+03
 5.38262914e+02 1.47396101e+04 7.06277029e-30 4.71736203e+03
 8.80327654e+03 1.02347985e+04 1.61790184e+04 2.47339704e+03
 3.31321580e-29 1.42524373e+04 1.28500554e+04 1.51855724e-29
 2.02145607e-30 1.24208899e+04 1.28438403e+04 4.89093761e-29
 2.37151310e-29 4.31716456e-30 6.67080503e-29 4.04861829e+03
 2.46605273e+04 1.01284708e+04 2.92006243e+03 9.51638650e+03
 8.26573508e+03 6.87124652e+03 2.56872832e-29 9.71505468e+03
 2.37915554e+04 4.65920972e-29 1.24730281e+04 2.27842191e+03
 9.03492256e-30 1.15798396e+04 2.87071414e-29 1.05009156e+04
 1.01185227e+04 2.25435491e+04 1.04169313e+04 1.77472768e+04
 1.74303619e+04 2.09527637e+04 1.30590790e+04 4.40481139e+03
 1.08098596e-29 3.48084874e-29 5.93397101e+03 1.61384438e+04
 2.01652569e-29 9.412096

In [43]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...



In [44]:
print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

658438.0857378774
4.8027824062118244e-27
4.8027824062118244e-27
658438.0857378774


In [45]:
print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC



In [46]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

658438.0857378774
4.8027824062118244e-27
4.8027824062118244e-27


In [47]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.7;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  70
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.7
   k/|V|: %.03f
 0.7
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

710832.4025112945
1.724647154039437e-28
1.724647154039437e-28
710832.4025112945
  - GNC



In [48]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[1, 3, 11, 13, 17, 19, 20, 25, 26, 28, 39, 40, 42, 43, 45, 46, 50, 52, 54, 65, 67, 68, 69, 80, 82, 91, 92, 94, 95, 99]
[ 1  3 11 13 17 19 20 25 26 28 39 40 42 43 45 46 50 52 54 65 67 68 69 80
 82 91 92 94 95 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
710832.4025112945
1.724647154039437e-28
1.724647154039437e-28


In [49]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.8;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  80
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.8
   k/|V|: %.03f
 0.8
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

706068.0528464385
5.789376227707141e-28
5.789376227707141e-28
706068.0528464385
  - GNC



In [50]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 4, 11, 15, 17, 19, 20, 26, 36, 48, 73, 75, 76, 80, 81, 83, 87, 91, 93, 99]
[ 0  4 11 15 17 19 20 26 36 48 73 75 76 80 81 83 87 91 93 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
706068.0528464385
5.789376227707141e-28
5.789376227707141e-28


In [51]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.9;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  90
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.9
   k/|V|: %.03f
 0.9
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

976283.552297634
1.7961376735750913e-28
1.7961376735750913e-28
976283.552297634
  - GNC



In [52]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[20, 38, 43, 47, 54, 67, 80, 91, 94, 98]
[20 38 43 47 54 67 80 91 94 98]
[ True  True  True  True  True  True  True  True  True  True]
976283.552297634
1.7961376735750913e-28
1.7961376735750913e-28


In [53]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

569896.3143943279
7.211831106776621e-27
7.211831106776621e-27
569896.3143943279
  - GNC



In [54]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[1, 12, 15, 20, 21, 23, 25, 26, 27, 30, 31, 34, 35, 36, 37, 38, 40, 42, 46, 49, 50, 51, 52, 54, 55, 57, 61, 67, 68, 69, 70, 75, 81, 82, 87, 88, 90, 92, 96, 98]
[ 1 12 15 20 21 23 25 26 27 30 31 34 35 36 37 38 40 42 46 49 50 51 52 54
 55 57 61 67 68 69 70 75 81 82 87 88 90 92 96 98]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
569896.3143943279
7.211831106776621e-27
7.211831106776621e-27


In [55]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.5;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
#results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  50
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.5
   k/|V|: %.03f
 0.5
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

502029.81877287827
1.1329282897857913e-27
1.1329282897857913e-27
502029.81877287827
  - GNC



In [56]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 16, 20, 24, 25, 34, 38, 39, 40, 42, 44, 45, 46, 47, 48, 50, 51, 53, 54, 60, 61, 66, 68, 72, 75, 76, 78, 79, 80, 83, 84, 85, 86, 87, 89, 91, 92, 93, 95, 96, 99]
[ 1  2  3  5  7  8  9 10 11 12 16 20 24 25 34 38 39 40 42 44 45 46 47 48
 50 51 53 54 60 61 66 68 72 75 76 78 79 80 83 84 85 86 87 89 91 92 93 95
 96 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
502029.81877287827
1.1329282897857913e-27
1.1329282897857913e-27
